In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [4]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'
ab_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\repo\\base'

In [2]:
#stationär
data_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\data'
ab_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\base'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


CONFIG

In [98]:
# Budget config file
include_pheno = False
threshold_year = 1970
data_path = data_dir
ab_path = ab_dir
max_length = [51,37]
mask_prob = 0.15
embedding_dim = 32
drop_prob = 0.2

#Encoder
enc_dim_inp = 32 
enc_dim_out = 32 
attention_heads = 8 

#BERT
num_encoders = 2

#trainer
epochs = 5
batch_size = 32
lr = 0.001
stop_patience = 10


Vocabulary

In [100]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
include_pheno = False
vocabulary = vocab_geno(NCBI, include_pheno)
vocab = vocab_pheno(ab_df)
print(len(vocabulary))
print(len(vocab))


1208
81


In [7]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from data_preprocessing import data_loader
from data_preprocessing import NCBIDataset

include_pheno = True
threshold_year = 1970

data_path = data_dir
ab_path = ab_dir

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)

max_length = [51,37]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [35]:
test_tensor = test_set[5][3]
print(test_tensor)
test_tensor = test_tensor[test_tensor != -1]
print(  test_tensor.tolist()  )

selected = pred_res[test_tensor.tolist() ]
print(selected)
expected = test_set[5][4]
expected = expected[expected != -1] 
print(expected)

tensor([16, 29, 72, 69, 25, 45, 78, 54,  2,  5, 66, 22, 42,  3, 30, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1])
[16, 29, 72, 69, 25, 45, 78, 54, 2, 5, 66, 22, 42, 3, 30]
tensor([1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.])
tensor([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])


In [9]:
from torch.utils.data import DataLoader 
from bert_builder import BERT_ft

loader = DataLoader(test_set, batch_size=16, shuffle=False)
for i, batch in enumerate(loader):
    input, token_target, attn_mask, AB_idx, SR_class  = batch
    bert_test = BERT_ft(vocab_size=len(vocabulary_geno), max_length=51, dim_embedding = 128, dim_hidden= 128, attention_heads=8, num_encoders=2, dropout_prob=0.2, num_ab=81, device=device)
    res_pred = bert_test.forward(input, attn_mask)

In [30]:
test = res_pred[14]
print(test)
pred_res = torch.where(test > 0, torch.ones_like(test), torch.zeros_like(test))
print(pred_res)
indicies = [1,2,3,4]
test = pred_res[indicies]
print(test)

tensor([ 1.0435,  0.9513,  0.3644, -0.0901, -0.7001, -0.0680, -0.1490, -0.0619,
        -0.1270, -0.8133, -0.1457,  0.2177, -0.6492,  0.1714, -0.4112, -0.3951,
         0.3892, -0.2889,  0.0415, -0.8135, -0.2810,  0.7298,  0.6785,  0.1533,
        -0.6075,  0.2196, -0.1743, -0.8824, -0.6209,  0.5175, -0.1205,  0.8606,
         0.2940,  0.5152,  0.5396,  0.3415, -0.6014, -0.4751,  0.0466, -0.4792,
         0.3310, -0.5104,  0.0227, -0.4278,  0.4219,  0.3178,  0.0415, -0.9904,
         0.5631, -0.3809,  0.6884,  0.0810,  1.2024,  0.3498,  0.1946,  0.8521,
         0.9092, -0.1204,  0.8237,  0.4910,  0.7333, -0.4589,  0.3011,  0.3034,
         0.3296, -0.0346, -0.5093,  0.8257, -0.2093,  0.0155, -0.4725,  0.4390,
        -0.1331,  1.2029, -0.1866,  0.4317,  0.7898,  0.6166, -0.8400,  0.4163,
        -0.3838], grad_fn=<SelectBackward0>)
tensor([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
        1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1.,

Trainer

In [ ]:
import numpy as np
from create_dataset import NCBIDataset
def get_split_indices(size_to_split, val_share, random_state: int = 42):
    indices = np.arange(size_to_split)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    train_share = 1 - val_share
    
    train_size = int(train_share * size_to_split)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    return train_indices, val_indices
max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

In [83]:
import time
from torch import nn
from torch.utils.data import DataLoader
import wandb
from pathlib import Path

class BertTrainer_pt:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, wandb_mode, project_name, wandb_name):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.01
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        
        self.wandb_mode = wandb_mode
        self.project_name = project_name
        self.wandb_name = wandb_name
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)

        self.device = device


    def __call__(self):   
        if self.wandb_mode:
            self._init_wandb()   
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss = self.train(self.current_epoch)
            self.train_losses.append(avg_epoch_loss) 
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses.append(val_results[0])  
            self.val_accs.append(val_results[1])
            if self.wandb_mode:
                self._report_epoch_results()
            self._report_epoch_results()
            criterion = self.stop_early()
            if criterion:
                print(f"Training interrupted at epoch: {self.current_epoch+1}")
                break

        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.current_epoch,
            "train_losses": self.train_losses,
            "val_losses": self.val_losses,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):
        self.train_losses = []
        self.val_losses = []
        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss = 0
        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask, AB_idx, SR_class  = batch
            
            self.optimizer.zero_grad() 

            tokens, res_pred = self.model(input, attn_mask) 
            loss = self.criterion(tokens.transpose(-1, -2), token_target) 
            
            epoch_loss += loss.item() 
            
            loss.backward() 
            self.optimizer.step()         
        avg_epoch_loss = epoch_loss / self.num_batches
        return avg_epoch_loss 
    
    def evaluate(self, loader):
        self.model.eval()
        epoch_loss = 0
        total_correct = 0
        total_tokens = 0

        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask = batch
                tokens = self.model(input, attn_mask)
                loss = self.criterion(tokens.transpose(-1, -2), token_target)
                epoch_loss += loss.item()
                
                token_mask =  token_target != -1
                predicted_tokens = tokens.argmax(dim=-1)
                token_target = torch.masked_select(token_target, token_mask)
                predicted_tokens = torch.masked_select(predicted_tokens, token_mask)
                
                correct = (predicted_tokens == token_target).sum().item()
                total_correct += correct
                total_tokens += token_target.numel() 
        
        avg_epoch_loss = epoch_loss / len(loader)
        accuracy = total_correct / total_tokens

        return avg_epoch_loss, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    def _init_wandb(self):
        self.wandb_run = wandb.init(
            project=self.project_name, # name of the project
            name=self.wandb_name, # name of the run
            
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "num_heads": self.model.attention_heads,
                "num_encoders": self.model.num_encoders,
                "emb_dim": self.model.dim_inp,
                'ff_dim': self.model.dim_out,
                "lr": self.lr,
                "weight_decay": self.weight_decay,
                "max_seq_len": self.model.max_length[0],
                "vocab_size": len(self.train_set.vocab),
                "num_parameters": sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            }
        )
        self.wandb_run.watch(self.model) # watch the model for gradients and parameters
        self.wandb_run.define_metric("epoch", hidden=True)
        self.wandb_run.define_metric("batch", hidden=True)

        self.wandb_run.define_metric("Losses/train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("Losses/val_loss", summary="min", step_metric="epoch")
        
        self.wandb_run.define_metric("Losses/final_val_loss")
        self.wandb_run.define_metric("Accuracies/final_val_acc")
        self.wandb_run.define_metric("final_epoch")

        return self.wandb_run
    
    def _report_epoch_results(self):
        wandb_dict = {
            "epoch": self.current_epoch+1,
            "Losses/train_loss": self.train_losses[-1],
            "Losses/val_loss": self.val_losses[-1],
            "Accuracies/val_acc": self.val_accs[-1],
        }
        self.wandb_run.log(wandb_dict)



class BertTrainer_ft:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, wandb_mode, project_name, wandb_name):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.01
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        
        self.wandb_mode = wandb_mode
        self.project_name = project_name
        self.wandb_name = wandb_name
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)

        self.device = device


    def __call__(self):   
        if self.wandb_mode:
            self._init_wandb()   
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss = self.train(self.current_epoch)
            self.train_losses.append(avg_epoch_loss) 
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses.append(val_results[0])  
            self.val_accs.append(val_results[1])
            if self.wandb_mode:
                self._report_epoch_results()
            self._report_epoch_results()
            criterion = self.stop_early()
            if criterion:
                print(f"Training interrupted at epoch: {self.current_epoch+1}")
                break

        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.current_epoch,
            "train_losses": self.train_losses,
            "val_losses": self.val_losses,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):
        self.train_losses = []
        self.val_losses = []
        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss = 0
        reporting_loss = 0
        printing_loss = 0
        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask = batch
            
            self.optimizer.zero_grad() 

            tokens = self.model(input, attn_mask) 
            loss = self.criterion(tokens.transpose(-1, -2), token_target) 
            
            epoch_loss += loss.item() 
            reporting_loss += loss.item()
            printing_loss += loss.item()
            
            loss.backward() 
            self.optimizer.step()         
        avg_epoch_loss = epoch_loss / self.num_batches
        return avg_epoch_loss 
    
    def evaluate(self, loader):
        self.model.eval()
        epoch_loss = 0
        total_correct = 0
        total_tokens = 0

        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask = batch
                tokens = self.model(input, attn_mask)
                loss = self.criterion(tokens.transpose(-1, -2), token_target)
                epoch_loss += loss.item()
                
                token_mask =  token_target != -1
                predicted_tokens = tokens.argmax(dim=-1)
                token_target = torch.masked_select(token_target, token_mask)
                predicted_tokens = torch.masked_select(predicted_tokens, token_mask)
                
                correct = (predicted_tokens == token_target).sum().item()
                total_correct += correct
                total_tokens += token_target.numel() 
        
        avg_epoch_loss = epoch_loss / len(loader)
        accuracy = total_correct / total_tokens

        return avg_epoch_loss, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    def _init_wandb(self):
        self.wandb_run = wandb.init(
            project=self.project_name, # name of the project
            name=self.wandb_name, # name of the run
            
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "num_heads": self.model.attention_heads,
                "num_encoders": self.model.num_encoders,
                "emb_dim": self.model.dim_inp,
                'ff_dim': self.model.dim_out,
                "lr": self.lr,
                "weight_decay": self.weight_decay,
                "max_seq_len": self.model.max_length[0],
                "vocab_size": len(self.train_set.vocab),
                "num_parameters": sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            }
        )
        self.wandb_run.watch(self.model) # watch the model for gradients and parameters
        self.wandb_run.define_metric("epoch", hidden=True)
        self.wandb_run.define_metric("batch", hidden=True)

        self.wandb_run.define_metric("Losses/train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("Losses/val_loss", summary="min", step_metric="epoch")
        
        self.wandb_run.define_metric("Losses/final_val_loss")
        self.wandb_run.define_metric("Accuracies/final_val_acc")
        self.wandb_run.define_metric("final_epoch")

        return self.wandb_run
    
    def _report_epoch_results(self):
        wandb_dict = {
            "epoch": self.current_epoch+1,
            "Losses/train_loss": self.train_losses[-1],
            "Losses/val_loss": self.val_losses[-1],
            "Accuracies/val_acc": self.val_accs[-1],
        }
        self.wandb_run.log(wandb_dict)

In [84]:
from bert_builder import BERT
from data_preprocessing import data_loader
from build_vocabulary import make_vocabulary
from misc import get_split_indices
from create_dataset import NCBIDataset

include_pheno = False

NCBI = data_loader(include_pheno,threshold_year,path)
vocabulary = make_vocabulary(NCBI, include_pheno)
reduced_samples = 1000
NCBI = NCBI.head(reduced_samples)
print(f"Data loaded, {len(NCBI)} samples found")
print(f"length  of vocabulary:",len(vocabulary))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob,include_pheno)

bert_test = BERT(len(vocabulary), max_length, enc_dim_inp, enc_dim_out, attention_heads, num_encoders, drop_prob)

test = BertTrainer(bert_test, train_set, val_set, epochs, batch_size, lr, device, stop_patience, True, "NCBI", "test")
test()

Data loaded, 1000 samples found
length  of vocabulary: 1208


Epoch 1/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:06
Epoch 2/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:12
Epoch 3/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:18
Epoch 4/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:24
Epoch 5/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:31
-=Training completed=-


{'best_epoch': 4,
 'train_losses': [6.716255130767823,
  5.805294799804687,
  5.029448509216309,
  4.463122625350952,
  4.1071098518371585],
 'val_losses': [6.11516353062221,
  5.36531925201416,
  4.712193965911865,
  4.294982944216047,
  4.067011458533151],
 'val_accs': [0.24260355029585798,
  0.23668639053254437,
  0.1893491124260355,
  0.1952662721893491,
  0.2485207100591716]}

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

------------------------------

In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()